In [10]:
import pandas as pd
import numpy as np
import vaex
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [15]:
traf = vaex.open('../../TRAFICO/traf_hist_unir.hdf5')

traf.info()

In [3]:
acci_filtered_vaex = vaex.open('../../acci_filtered_vaex.hdf5')
accis_id = acci_filtered_vaex.to_pandas_df()

ids = accis_id.idelem.value_counts().index

In [4]:
ids

Index(['4206', '4436', '4774', '3901', '4452', '5828', '4170', '3798', '5484',
       '4166',
       ...
       '5217', '6608', '5388', '6266', '6420', '10443', '10406', '10584',
       '10462', '5426'],
      dtype='object', length=4371)

In [5]:
#uniques in traf idelem

traf_ids = traf.idelem.unique()

In [7]:
#list of ids that are in accis and in traf

ids_in_accis = list(set(ids).intersection(traf_ids))

In [12]:
acci = pd.read_csv('../data/biblio_acci.csv')

In [31]:
import tqdm

for id in tqdm.tqdm(ids_in_accis[64:]):

    #filtrar por idelem

    try:
    
        traf1 = traf[traf.idelem == id]

        traf2 = traf1.to_pandas_df()

        traf_acci = pd.merge(traf2, acci, on='unir', how='left')

        traf_acci['accidente'] = traf_acci['accidente'].fillna(0)

        if traf_acci['accidente'].sum() > 0:
        
            traf_acci.drop(['unir', 'idelem'], axis=1, inplace=True)

            traf_acci['fecha'] = traf_acci['fecha_full'].str.split(' ').str[0]
            
            traf_acci['hora'] = traf_acci['fecha_full'].str.split(' ').str[1]

            traf_acci.drop('fecha_full', axis=1, inplace=True)

            traf_acci['prec'] = pd.to_numeric(traf_acci['prec'].str.replace(',', '.'), errors='coerce')

            traf_acci['velmedia'] = pd.to_numeric(traf_acci['velmedia'].str.replace(',', '.'), errors='coerce')

            traf_acci.fillna(0, inplace=True)

            traf_acci['mes'] = traf_acci['fecha'].str.split('-').str[1]

            traf_acci['fecha'] = pd.to_datetime(traf_acci['fecha'])

            traf_acci['dia_semana'] = traf_acci['fecha'].dt.dayofweek

            traf_acci.drop('fecha', axis=1, inplace=True)

            traf_acci['dia_semana'] = traf_acci['dia_semana'].astype('object')

                #get dummies

            traf_acci_dum = pd.get_dummies(traf_acci, columns=['mes', 'dia_semana', 'hora'])

                #Xtest, ytest

            X = traf_acci_dum.drop('accidente', axis=1)

            y = traf_acci_dum['accidente']

            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

                #scaler

            scaler = StandardScaler()

            X_train = scaler.fit_transform(X_train)

            X_test = scaler.transform(X_test)

                #model

            model = LogisticRegression()
            model.fit(X_train, y_train)

            filename = 'modelo_' + id + '.sav'
            pickle.dump(model, open('../modelos/' + filename, 'wb'))#generar tabla para modelo

    except:

        continue




  0%|          | 0/4307 [00:00<?, ?it/s]c:\Users\calba\anaconda3\lib\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)
c:\Users\calba\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
  0%|          | 1/4307 [00:26<31:33:34, 26.39s/it]c:\Users\calba\anaconda3\lib\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not inf

In [30]:
+

1